In [5]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
api_key_pine = os.environ["PINE_API_KEY"]

In [6]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [4]:
from pinecone import Pinecone, ServerlessSpec
index_name = "langchain-hybrid-search-pinecone"

e:\Langchain-Complete\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [15]:
index=pc.Index(index_name)
index

In [7]:
pc=Pinecone(api_key=api_key_pine)
#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(name=index_name,
    dimension=384, #dimensions of dense vectors
    metric='dotproduct', # spars value supported only for dot product
    spec=ServerlessSpec(cloud='aws',region="us-east-1")
    )

In [8]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings


In [9]:
embeddings =HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

e:\Langchain-Complete\venv\lib\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
from pinecone_text.sparse import  BM25Encoder
bm25encoder = BM25Encoder().default()
bm25encoder

In [12]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')  # Required for tokenization


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Armghan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Armghan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Armghan/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [10]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print("Stopwords loaded successfully:", len(stop_words))


Stopwords loaded successfully: 179


In [13]:
# First ensure NLTK data is properly downloaded
import nltk

# Set custom download path (use forward slashes to avoid escape issues)
nltk_dir = "C:/Users/Armghan/nltk_data"
os.makedirs(nltk_dir, exist_ok=True)

# Download required NLTK data with explicit path
nltk.download('stopwords', download_dir=nltk_dir)
nltk.download('punkt', download_dir=nltk_dir)

# Add to NLTK path and set environment variable
nltk.data.path.append(nltk_dir)
os.environ['NLTK_DATA'] = nltk_dir

# Now proceed with BM25Encoder
from pinecone_text.sparse import BM25Encoder

bm25encoder = BM25Encoder()

sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "The dog barks at the fox.",
    "The fox runs away from the dog.",
    "The fox and the dog play fetch.",
    "The fox and the dog play fetch in the park.",
    "The fox and the dog play fetch in the park with a ball."
]

# Verify NLTK resources are accessible
try:
    from nltk.corpus import stopwords
    stopwords.words('english')
    nltk.word_tokenize("test sentence")
except LookupError as e:
    print(f"NLTK resource missing: {e}")
    # If still failing, use temporary directory
    nltk.download('stopwords')
    nltk.download('punkt')

# Now fit the encoder
bm25encoder.fit(sentences)
bm25encoder.dump("bm25encoder_values.json")
bm25encoder = BM25Encoder().load("bm25encoder_values.json")
print("BM25 Encoder successfully fitted and loaded!")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:/Users/Armghan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:/Users/Armghan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 6/6 [00:00<00:00, 3007.75it/s]

BM25 Encoder successfully fitted and loaded!


In [17]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25encoder,index=index)

In [18]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x00000212ABC37760>, index=<pinecone.data.index.Index object at 0x00000212940D8490>)

In [19]:
retriever.add_texts([
    "The quick brown fox jumps over the lazy dog.",
    "The dog barks at the fox.",
    "The fox runs away from the dog.",
    "The fox and the dog play fetch.",
    "The fox and the dog play fetch in the park.",
    "The fox and the dog play fetch in the park with a ball."
])

100%|██████████| 1/1 [00:04<00:00,  4.87s/it]


In [21]:
# Query the retriever
query = "what does the fox do?"
results = retriever.invoke(query)

# Print the results
for result in results:
    print(result)

page_content='The fox and the dog play fetch.'
page_content='The dog barks at the fox.'
page_content='The fox runs away from the dog.'
page_content='The quick brown fox jumps over the lazy dog.'


In [22]:
query = "What does the dog do?"
results = retriever.invoke(query)
for result in results:
    print(result)

page_content='The dog barks at the fox.'
page_content='The fox and the dog play fetch.'
page_content='The quick brown fox jumps over the lazy dog.'
page_content='The fox runs away from the dog.'


In [23]:
query = "Where do the fox and the dog play fetch?"
results = retriever.invoke(query)
for result in results:
    print(result)

page_content='The fox and the dog play fetch.'
page_content='The fox and the dog play fetch in the park.'
page_content='The fox and the dog play fetch in the park with a ball.'
page_content='The quick brown fox jumps over the lazy dog.'


In [24]:
query = "How does the fox react to the dog?"
results = retriever.invoke(query)
for result in results:
    print(result)

page_content='The fox and the dog play fetch.'
page_content='The dog barks at the fox.'
page_content='The fox runs away from the dog.'
page_content='The fox and the dog play fetch in the park.'


In [25]:
query = "What happens in the park?"
results = retriever.invoke(query)
for result in results:
    print(result)

page_content='The fox and the dog play fetch in the park.'
page_content='The fox and the dog play fetch in the park with a ball.'
page_content='The fox and the dog play fetch.'
page_content='The quick brown fox jumps over the lazy dog.'


In [29]:
query = "What do the fox and the dog do together?"
results = retriever.invoke(query)
for result in results:
    print(result)

page_content='The fox and the dog play fetch.'
page_content='The dog barks at the fox.'
page_content='The quick brown fox jumps over the lazy dog.'
page_content='The fox runs away from the dog.'


In [30]:
retriever.add_texts([
    "The fox is known for its cunning nature.",
    "The dog is loyal and protective.",
    "In the forest, the fox and the dog often cross paths."
])

100%|██████████| 1/1 [00:01<00:00,  1.50s/it]


In [31]:
query = "Why does the fox run away from the dog?"
results = retriever.invoke(query)
for result in results:
    print(result)

page_content='The fox runs away from the dog.'
page_content='The fox and the dog play fetch.'
page_content='In the forest, the fox and the dog often cross paths.'
page_content='The dog barks at the fox.'
